In [1]:
import sys
sys.path.append("C:/Users/Administrator/Jupyter/Libraries")
sys.path.append("C:/Users/Administrator/Jupyter/Libraries/dataio_stanford.py")

import pandas as pd
import numpy as np
from uuid import UUID
import re
from datetime import date, datetime
import time
from pathlib import Path
import dataio_stanford
from pdm import pull
import geo
import date
import dahelp as da
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
def geocode(df):
    con = da.conn.hippo_sql()
    locations = da.pd.read_sql('SELECT * FROM CHARGE_LOCATIONS;', con)
    df['CrimeCase'] = df['CrimeCase'].astype(str)
    locations = locations[['CrimeCase', 'charge_sequence', 'gc_street_address', 'gc_zip_code', 'gc_lat', 'gc_lon']]
    df = da.pd.merge(df, locations, how='left', on=['CrimeCase', 'charge_sequence'])
    
    df['incident_lat_lon'] = df.gc_lat.astype(str) + ',' + df.gc_lon.astype(str)
    return df

def check_priors(priors, row):
    #Prior Arrest Flag
        if pd.notnull(row.arrest_date):
            priors[0] = 1
        
        #Prior Indictment Flag
        if row.charge_method == 'Grand Jury Indictment':
            priors[1] = 1
        
        #Prior Conviction Flag
        if row.case_disposition == 'Guilty':
            priors[2] = 1
        
        #Prior Violent Arrest Flag
        if pd.notnull(row.arrest_date)  and row.case_is_violent == True:
            priors[3] = 1
        
        #Prior Violent Indictment Flag
        if row.charge_method == 'Grand Jury Indictment' and row.case_is_violent == True:
            priors[4] = 1
        
        #Prior Violent Conviction Flag
        if row.case_disposition == 'Guilty' and row.case_is_violent == True:
            priors[5] = 1
        
        #Prior Felony Arrest Flag
        if pd.notnull(row.arrest_date) and row.case_has_felony == True:
            priors[6] = 1
        
        #Prior Felony Indictment Flag
        if row.charge_method == 'Grand Jury Indictment' and row.case_has_felony == True:
            priors[7] = 1
        
        #Prior Felony Conviction Flag
        if row.case_disposition == 'Guilty' and row.case_has_felony == True:
            priors[8] = 1
            
        return(priors)




In [3]:
%%time
df = dataio_stanford.pull_cms_master()
print(df.shape)

C:\Users/Administrator/Jupyter/Libraries\dataio_stanford.py:41: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(q, cms)


(1761811, 142)
CPU times: total: 8min 25s
Wall time: 25min 15s


In [15]:
%%time
pdm_df = da.pdm.pull()

CPU times: total: 2.83 s
Wall time: 9.47 s


In [16]:
pdm_df

,pdm_cms_case_num,pdm_number,pdm_def_last,pdm_def_first,pdm_def_dob,pdm_filedate,pdm_heardate,pdm_ada,pdm_judge,pdm_outcome,pdm_metro_num,pdm_psa_flag,pdm_psa_nca,pdm_psa_fta,pdm_psa_na,pdm_firearm,pdm_fed_ref,pdm_psa_recommendation
0,2017-00403-1,D-202-LR-2017-00008,Fazio,Genoveva,1980-03-29,2017-01-19,2017-01-24,John Litchford,"Whitaker, Stanley",Denied,T-4-FR-2016-006786,,NaN,NaN,FALSE,FALSE,FALSE,Release
1,2017-00405-1,D-202-LR-2017-00009,Heh,Christopher,1985-05-28,2017-01-19,2017-01-24,Tim Callaway,"Whitaker, Stanley",Denied,T-4-FR-2017-000348,,NaN,NaN,TRUE,FALSE,FALSE,Release
2,2017-00244-2,D-202-LR-2017-00004,Lucero,Reymundo,1995-08-03,2017-01-13,2017-01-24,Les Romaine,"Brown, Charles W.",Denied,T-4-FR-2017-000236,,NaN,NaN,TRUE,FALSE,FALSE,Release
3,2017-00317-1,D-202-LR-2017-00010,Groves,Elexus,1995-02-27,2017-01-23,2017-01-26,Guinevere Ice,"Brown, Charles W.",Denied,T-4-FR-2017-000399,,NaN,NaN,TRUE,FALSE,FALSE,Release
4,2017-00364-1-DV,D-202-LR-2017-00006,Rowley,Shawn,1985-02-24,2017-01-19,2017-01-26,Consuelo Garcia,"Leos, Cindy",Denied,T-4-FR-2017-000319,,NaN,NaN,TRUE,FALSE,FALSE,Release
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6522,2022-05465-1,D-202-PD-2022-00860,Archuleta,Antonio Marcos,1988-02-22,2022-09-19,2022-09-21,Peter Haynes,"Loveless, Brett R.",Denied,T-4-FR-2022-004515,FALSE,4,3,NaN,TRUE,,Release
6523,2022-05545-1,D-202-PD-2022-00861,Campos,Morandy L.,1999-03-03,2022-09-21,2022-09-23,Emilie Edmonds-Blevins,"Weaks, Courtney",Granted,T-4-FR-2022-004531,FALSE,1,1,NaN,FALSE,,Release
6524,2022-05522-1,D-202-PD-2022-00862,Henry,Michael Paul,1994-03-30,2022-09-20,2022-09-22,Graham Conlon,"Weaks, Courtney",Denied,T-4-FR-2022-004563,FALSE,3,2,NaN,TRUE,,Release
6525,2022-05596-1,D-202-PD-2022-00863,Lovato,Joseph Manuel,1982-05-14,2022-09-20,2022-09-22,Philemon Lamont Norris,"Weaks, Courtney",Granted,T-4-FR-2022-004565,TRUE,6,6,NaN,FALSE,,Detain


In [17]:
%%time
df = pd.merge(df, pdm_df, left_on = 'cms_case_num', right_on='pdm_cms_case_num', how='left')

CPU times: total: 24.3 s
Wall time: 24.9 s


In [6]:
%%time
df = geocode(df)
df.to_csv('stanford_data_pull', index=False)

CPU times: total: 4min 42s
Wall time: 4min 43s


In [ ]:
pd.set_option('display.max_rows', None)
dtype_dict = df.dtypes.to_dict()

In [ ]:
dtype_dict

In [21]:
df.shape

(1761970, 165)

In [20]:
%%time
#df2=pd.read_csv('stanford_data_pull', parse_dates=False, low_memory=False)
df2 = df.sort_values(by=['cms_defendant_guid', 'open_date', 'charge_sequence'], ignore_index=True)

CPU times: total: 40.6 s
Wall time: 40.7 s


In [22]:
df2.shape

(1761970, 165)

In [23]:
df2 = df2.drop(labels = ['pdm_cms_case_num','pdm_number','pdm_def_last','pdm_def_first','pdm_def_dob',
                         'pdm_filedate','pdm_heardate','pdm_ada','pdm_judge','pdm_outcome','pdm_metro_num',
                         'pdm_psa_flag','pdm_psa_nca','pdm_psa_fta','pdm_firearm','pdm_fed_ref',
                         'pdm_psa_recommendation','pdm_psa_na','charge_enhancement','victim_name','victim_race',
                         'victim_ethnicity','victim_gender','victim_dob','victim_relationship',
                         'gc_street_address','gc_zip_code','gc_lat','gc_lon','incident_lat_lon'],  axis=1)
df2.drop_duplicates(inplace=True, ignore_index=True)
df2.shape

(1735950, 135)

In [ ]:
df2['CrimeCase'][df2.charge_sequence == 1].value_counts()

In [27]:
pd.set_option('display.max_columns', None)

df2[df2.CrimeCase == '799b65b5-90a5-4b13-9908-59d3998ac699']

,CrimeCase,arraign_date,arrest_date,closed_date,complaint_date,custody_date,filed_dist_date,juvenile_detained_date,open_date,time_to_file,time_to_close,calendar_open_year,calendar_closed_year,calendar_open_month,calendar_closed_month,fiscal_open_year,fiscal_closed_year,custody_type,custody_location,cms_case_class,cms_case_type,crime_division,cms_case_stage,case_sealed,case_rank,photo_evidence,video_evidence,audio_evidence,forfeiture_evidence,drug_tested,finger_printed,dwi_breath_tested,dwi_blood_tested,dna_tested,other_report_type,cms_case_num,dist_docket_num,mag_docket_num,case_state_tracking_num,additional_case_num,referring_agency,referring_agency_case_num,referring_agent_guid,referring_agent_name,cms_defendant_guid,defendant_dob,defendant_last_name,defendant_first_name,defendant_middle_name,defendant_name,defendant_ssn,defendant_state_id,defendant_last_zip,defendant_race,defendant_ethnicity,defendant_gender,prosecutor_guid,prosecutor_name,defense_guid,defense_name,victim_advocate_guid,victim_advocate_name,investigator_guid,investigator_name,screener_guid,screener_name,district_judge_guid,dist_judge_name,magistrate_judge_guid,mag_judge_name,case_stage,case_category,weapon_used_flag,weapon_type,diversion_referral_date,diversion_apply_date,diversion_accept_date,diversion_reject_date,diversion_begin_date,diversion_end_date,diversion_completed_date,pretrial_diversion_flag,diversion_acceptance_flag,diversion_completion_flag,warrant_issue_date,case_has_felony,status,charge_trajectory_disposition,charge_is_murder,charge_is_aggravated_assault,charge_is_armed_robbery,charge_is_rape,charge_is_felony,charge_is_domestic_violence,charge_reason,charge_cms_disposition,charge_method,charge_sequence,charge_description,charge_master_statute,charge_code,charge_code_is_active,charge_degree,charge_location,charge_start_date,charge_end_date,charge_primary,drug_type,plea_offered,plea_date,sentence_date,sentence_days,sentence_severity,charge_is_probation_violation,sentence_is_concurrent,sentence_is_consecutive,sentence_is_death,sentence_length,suspended_length,probation_length,fines,fines_suspended,restitution,charge_trajectory,charge_disposition,case_is_murder,case_is_aggravated_assault,case_is_armed_robbery,case_is_rape,case_trajectory,case_disposition,case_is_juvenile,case_is_violent,case_is_difficult_victim,case_is_minor_victim
774154,799b65b5-90a5-4b13-9908-59d3998ac699,2004-08-11,2004-07-16,2007-03-06 08:50:00,2004-07-16,NaT,NaT,NaT,2004-07-22,NaN,957.0,2004.0,2007.0,Jul,Mar,2005.0,2007.0,NaN,None,None,Magistrate Court,Metro Court Division,Closed,None,NaN,False,False,False,False,None,None,None,None,None,None,DV-02589-04,None,DV-02589-04,Not Available,None,Albuquerque Police Department,04-77737,8170f079-4f15-4a65-a998-919f6614a1af,LEE SPEER,71031ae3-0808-474d-a317-f70c5ba0f5a2,1982-11-13,Moss,Louis,C.,LOUIS MOSS,558-73-9072,None,87108,None,None,F,b2796577-3023-4e0d-a5f2-de30a1c5a6e9,CLEMENT ROGERS,None,,None,,None,,None,,None,,62b63ac0-f416-4bb5-990c-c9ed2074eec2,SANDRA ENGEL,Closed,Original Crime Case,None,None,NaT,NaT,NaT,NaT,NaT,NaT,NaT,None,None,None,NaT,False,Launched,Dismissal;Prosecutor Dismissed,False,False,False,False,False,False,Victim/Witness Uncooperative,Nolle Pros/Dismissed by Prosecutor,Criminal Complaint,1,BATTERY ON A HOUSEHOLD MEMBER,None,13596,False,None,None,2004-07-16,NaT,True,None,None,2007-03-02,NaT,0.0,None,None,False,False,False,"0 years, 0 months, and 0days","0 years, 0 months, and 0 days","0 years, 0 months, and 0 days",0.0,0.0,0.0,Dismissal,Prosecutor Dismissed,False,False,False,False,Dismissal,Prosecutor Dismissed,False,False,False,False
774156,799b65b5-90a5-4b13-9908-59d3998ac699,2004-08-11,2004-07-16,2007-03-06 08:50:00,2004-07-16,NaT,NaT,NaT,2004-07-22,NaN,957.0,2004.0,2007.0,Jul,Mar,2005.0,2007.0,NaN,None,None,Magistrate Court,Metro Court Division,Closed,None,NaN,False,False,False,False,None,None,None,None,None,None,DV-02589-04,None,DV-02589-04,Not Available,None,Albuquerque Police Department,04-77737,8170f079

In [26]:
df2.drop([774153, 774155], inplace=True)

In [40]:
%%time
flag_df = df2[df2.charge_sequence==1].reset_index(drop=True)
flag_df['prior_arrest'] = False
flag_df['prior_indictment'] = False
flag_df['prior_conviction'] = False
flag_df['prior_violent_arrest'] = False
flag_df['prior_violent_indictment'] = False
flag_df['prior_violent_conviction'] = False
flag_df['prior_felony_arrest'] = False
flag_df['prior_felony_indictment'] = False
flag_df['prior_felony_conviction'] = False

CPU times: total: 14.5 s
Wall time: 14.5 s


In [42]:
%%time
priors_labels = ['prior_arrest', 'prior_indictment', 'prior_conviction',
                 'prior_violent_arrest', 'prior_violent_indictment', 'prior_violent_conviction',
                 'prior_felony_arrest', 'prior_felony_indictment', 'prior_felony_conviction']
prior_def = np.nan
counter = 0 
for i, row in flag_df.iterrows():
    current_def = row.cms_defendant_guid
    
    if current_def != prior_def:
        priors = [0,0,0,0,0,0,0,0,0]   
    
    flag_df.loc[i, priors_labels] = np.where(priors, True, False)

    priors = check_priors(priors, row)
    prior_def = current_def
    
    if i%10000==0: print()
    
    counter+=1
    if counter==1000: break

KeyboardInterrupt: 

In [ ]:
%%time
merge_df = flag_df[['CrimeCase', 'prior_arrest', 'prior_indictment',
       'prior_conviction', 'prior_violent_arrest', 'prior_violent_indictment',
       'prior_violent_conviction', 'prior_felony_arrest',
       'prior_felony_indictment', 'prior_felony_conviction']]
final_df = df.merge(merge_df, how='left', on='CrimeCase')
final_df.to_csv(f'stanford_data_pull_final_{date.today()}')

In [ ]:
final_df.head(100)

In [ ]:
final_df.shape

In [51]:
%%time
priors_labels = ['prior_arrest', 'prior_indictment', 'prior_conviction',
                 'prior_violent_arrest', 'prior_violent_indictment', 'prior_violent_conviction',
                 'prior_felony_arrest', 'prior_felony_indictment', 'prior_felony_conviction']
flag_df[priors_labels].head()
priors=[0,1,0,0,1,1,0,0,1]
flag_df.loc[0, priors_labels] = np.where(priors, True, False)
flag_df[priors_labels].head()

CPU times: total: 328 ms
Wall time: 328 ms


,prior_arrest,prior_indictment,prior_conviction,prior_violent_arrest,prior_violent_indictment,prior_violent_conviction,prior_felony_arrest,prior_felony_indictment,prior_felony_conviction
0,False,True,False,False,True,True,False,False,True
1,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False


In [43]:
ids = flag_df.prior_arrest.isna()

In [ ]:
ids